<a href="https://colab.research.google.com/github/mariddc/Projeto-RL-Highway-gym/blob/main/new_HIGHWAY_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GYM - STABLE BASELINES
##  🏎 Ambiente Highway 🏎

O primeiro passo é instalar as bibliotecas necessárias para o notebook.

In [ ]:
# Install environment and agent
!pip install highway-env
# TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
!pip install git+https://github.com/carlosluis/stable-baselines3@75fd27e019aaf4dad612823b4544620df2c47844

In [ ]:
# Instalação de bibliotecas necessárias para visualizar o ambiente no Colab
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Agora vamos importar o ambiente do _gym_ e diferentes tipos de agentes que queremos treinar da biblioteca _stable baselines_. Além disso, também importamos ferramentas para, mais tarde, avaliarmos o desempenho do agente.

In [ ]:
# Environment
import gym
import highway_env

# Agent
from stable_baselines3 import PPO, DQN

# Evaluation
from stable_baselines3.common.evaluation import evaluate_policy 

import itertools

Configurando as ferramentas para visualização dos resultados.

In [4]:
import pyvirtualdisplay

# Cria um display dentro do Colab
_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

Criamos o ambiente que queremos, consultando a biblioteca do _gym_ e utilizando uma semente para garantir sempre os mesmos resultados.

In [ ]:
env = gym.make("racetrack-v0")
env.seed(73)

In [ ]:
from gym.wrappers import RecordVideo

env = RecordVideo(env, video_folder="videos")
env.unwrapped.set_record_video_wrapper(env)

In [ ]:
# Setting model
model = PPO('MlpPolicy', env, seed=32, verbose=1)

obs, done = env.reset(), False

for i in range(100):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

env.close()
'''
while not done: 
  for t in itertools.count():
    action, _states = model.predict(obs)

    obs, reward, done, info = env.step(action)

env.close()'''

In [9]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('videos/rl-video-episode-0.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


In [ ]:
# evaluation
eval_env = gym.make("racetrack-v0")
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

print(f"Recompensa Média: {mean_reward:.2f} +/- {std_reward}")

In [ ]:
# Setting new model for training
training_model = PPO('MlpPolicy', env, seed=32, verbose=1)

In [ ]:
training_model.learn(total_timesteps=10000)